In [89]:
import pandas as pd

In [90]:
df=pd.read_csv("fifa_eda_stats.csv")
df.head()

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,Left,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,20801,Cristiano Ronaldo,33,Portugal,94,94,Juventus,€77M,€405K,Right,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,190871,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,Right,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,193080,De Gea,27,Spain,91,93,Manchester United,€72M,€260K,Right,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,192985,K. De Bruyne,27,Belgium,91,92,Manchester City,€102M,€355K,Right,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [91]:
#Reset index and remove unnecessary columncs
df.set_index('ID', inplace=True)
df.drop(columns=['Jersey Number', 'Release Clause', 'Loaned From', 'Contract Valid Until' , ], inplace=True)

In [92]:
#Convert currency columns
def ConvertNumeric(value):
    # Remove Euro sign and whitespace
    value = value.replace('€', '').strip()

    # Check if value ends with 'M' (for millions)
    if value.endswith('M'):
        value = float(value[:-1]) * 1_000_000
    # Check if value ends with 'K' (for thousands)
    elif value.endswith('K'):
        value = float(value[:-1]) * 1_000
    else:
        # Otherwise convert to float
        value = float(value)
    
    return value

# Convert the columns
df['Value'] = df['Value'].apply(ConvertNumeric)
df['Wage'] = df['Wage'].apply(ConvertNumeric)

In [93]:
#Convert height column
def height_to_inches(height):
    # Check if  height is string 
    if isinstance(height, str):
        # Split string by apostrophe (') to feet and inches
        feet, inches = height.split("'")
        total_inches = int(feet) * 12 + int(inches.replace('"', ''))  # Convert to total inches
        return total_inches
    elif isinstance(height, (int, float)):
        # If numeric value, return directly
        return height
    else:
        # Uunexpected values, return NaN
        return None

# Convert Height column
df['Height'] = df['Height'].apply(height_to_inches)

In [94]:
#Convert weight column
def weight_to_float(weight):
    # Check if weight is string containing 'lbs'
    if isinstance(weight, str) and 'lbs' in weight:
        # Remove 'lbs' and convert to a float
        return float(weight.replace('lbs', '').strip())
    # If weight is numeric value, return as is
    elif isinstance(weight, (int, float)):
        return float(weight)
    else:
        return None  
    
# Convert weight column
df['Weight'] = df['Weight'].apply(weight_to_float)

In [95]:
#Drop players without statictics
df.dropna(subset=['Preferred Foot'], inplace=True)
#Drop players without clubs because they have no payment into
df.dropna(subset=['Club'], inplace=True)

In [96]:
#Create new column combinations
# Attacking = Crossing + Finishing + HeadingAccuracy + ShortPassing + Volleys
df['Attacking'] = df['Crossing'] + df['Finishing'] + df['HeadingAccuracy'] + df['ShortPassing'] + df['Volleys']

# Skill = Dribbling + Curve + FKAccuracy + LongPassing + BallControl
df['Skill'] = df['Dribbling'] + df['Curve'] + df['FKAccuracy'] + df['LongPassing'] + df['BallControl']

# Movement = Acceleration + SprintSpeed + Agility + Reactions + Balance
df['Movement'] = df['Acceleration'] + df['SprintSpeed'] + df['Agility'] + df['Reactions'] + df['Balance']

# Power = ShotPower + Jumping + Stamina + Strength + LongShots
df['Power'] = df['ShotPower'] + df['Jumping'] + df['Stamina'] + df['Strength'] + df['LongShots']

# Mentality = Aggression + Interceptions + Positioning + Vision + Penalties + Composure
df['Mentality'] = df['Aggression'] + df['Interceptions'] + df['Positioning'] + df['Vision'] + df['Penalties'] + df['Composure']

# Defending = Marking + StandingTackle + SlidingTackle
df['Defending'] = df['Marking'] + df['StandingTackle'] + df['SlidingTackle']

# Goalkeeping = GKDiving + GKHandling + GKKicking + GKPositioning + GKReflexes
df['Goalkeeping'] = df['GKDiving'] + df['GKHandling'] + df['GKKicking'] + df['GKPositioning'] + df['GKReflexes']

# Create a new column for all stats combined
df['AllStats'] = (df['Attacking'] + df['Skill'] + df['Movement'] + df['Power'] + 
                  df['Mentality'] + df['Defending'] + df['Goalkeeping'])

In [97]:
# Change 'Right' to 1 and 'Left' to 0
df['Preferred Foot'] = df['Preferred Foot'].map({'Right': 1, 'Left': 0}).fillna(0)

In [98]:
# Converte 'joined' column to datetime and calculating the number of days
# 'Joined' column is datetime 
df['Joined'] = pd.to_datetime(df['Joined'], dayfirst=False, errors='coerce')

# Target date (06-Jun-20) to datetime
target_date = pd.to_datetime('06-Jun-20', format='%d-%b-%y')

# Calculate the number of days between 'joined' and the target date
df['Days Joined'] = (target_date - df['Joined']).dt.days


In [99]:
# Split 'WorkRate' column on '/' and create two new columns
df[['AttackingWorkRate', 'DefensiveWorkRate']] = df['Work Rate'].str.strip().str.split('/', expand=True)

# Strip spaces from 'DefensiveWorkRate' and 'AttackingWorkRate' columns (if any) to ensure exact matching
df['DefensiveWorkRate'] = df['DefensiveWorkRate'].str.strip()
df['AttackingWorkRate'] = df['AttackingWorkRate'].str.strip()

# Replace 'Low', 'Medium', 'High' with numeric equivalents for both columns
df['AttackingWorkRate'] = df['AttackingWorkRate'].replace({'Low': 0, 'Medium': 1, 'High': 2})
df['DefensiveWorkRate'] = df['DefensiveWorkRate'].replace({'Low': 0, 'Medium': 1, 'High': 2})

# Convert the columns to integers
df['AttackingWorkRate'] = df['AttackingWorkRate'].astype(int)
df['DefensiveWorkRate'] = df['DefensiveWorkRate'].astype(int)

# Display the updated columns
df[['AttackingWorkRate', 'DefensiveWorkRate']]

C:\Users\mkori\AppData\Local\Temp\ipykernel_16768\3585391432.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['AttackingWorkRate'] = df['AttackingWorkRate'].replace({'Low': 0, 'Medium': 1, 'High': 2})
C:\Users\mkori\AppData\Local\Temp\ipykernel_16768\3585391432.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['DefensiveWorkRate'] = df['DefensiveWorkRate'].replace({'Low': 0, 'Medium': 1, 'High': 2})


,AttackingWorkRate,DefensiveWorkRate
ID,,
158023,1,1
20801,2,0
190871,2,1
193080,1,1
192985,2,2
...,...,...
238813,1,1
243165,1,1
241638,1,1


In [100]:
body_type_counts = df['Position'].value_counts()

# Display the result
print(body_type_counts)

Position
ST     2130
GK     1992
CB     1754
CM     1377
LB     1305
RB     1268
RM     1114
LM     1086
CAM     948
CDM     936
RCB     652
LCB     637
LCM     389
RCM     387
LW      374
RW      365
RDM     246
LDM     239
LS      206
RS      201
RWB      87
LWB      78
CF       74
LAM      21
RAM      21
RF       16
LF       15
Name: count, dtype: int64


In [101]:
df.to_csv('updated_player_data.csv', index=True)